# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [ ]:
import pandas as pd

## imports for Python, Pandas

In [ ]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [ ]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [ ]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [ ]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [ ]:
# load json as string
json.load((open('data/world_bank_projects_less.json')))

In [ ]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

## Load the JSON data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from pandas.io.json import json_normalize

In [ ]:
# specify the path to the JSON file
filename = 'data/world_bank_projects.json'

# load the JSON data into a list of dicts
with open(filename, 'r') as json_file:
    json_data = json.load(json_file)

## Solution to Task 1

In [ ]:
# Create a DataFrame, without using normalization
df = pd.read_json(filename)

In [ ]:
# count the number of rows for each countryname value
num_projects_by_countryname = df['countryname'].value_counts()

# convert the Series back into a DataFrame for readability
num_projects_by_countryname =  num_projects_by_countryname.to_frame('count').rename_axis('countryname')

In [ ]:
# print the first 10 rows.
solution_1 = num_projects_by_countryname.head(10)
print(solution_1)

In [ ]:
# make a bar chart of the top 10 country names
solution_1.plot(kind='bar',title='Frequency Distribution of Top 10 Country Names',legend=None)
plt.xlabel('Country Name')
plt.ylabel('Frequency')
plt.xticks(rotation=60,ha='right')
plt.show()

## Solution to Task 2

In [ ]:
# create a DataFrame from nested elements in mjtheme_namecode
df_2 = json_normalize(json_data, 'mjtheme_namecode')

In [ ]:
# count the number of rows for each code, print the top 10 results.  
solution_2 = df_2['code'].value_counts().to_frame('count').rename_axis('code').head(10)
print(solution_2)

In [ ]:
# create a bar chart of the top 10 theme codes
solution_2.plot(kind='bar',title='Frequency Distribution of Top 10 Major Project Theme Codes',legend=None)
plt.xlabel('Code')
plt.ylabel('Frequency')
plt.xticks(rotation='horizontal')
plt.show()

## Solution to Task 3

In [ ]:
# Create a DataFrame, without using normalization
df_3 = pd.read_json(filename)

In [ ]:
# create a dict with mjtheme codes as keys and non-empty names as values.
names = df_2[df_2.name != '']
names = names.drop_duplicates().set_index('code')
dict_mjthemes = names['name'].to_dict()

In [ ]:
# print the dict that maps codes to names.  NOTE: keys are strings
dict_mjthemes

In [ ]:
# create a custom function to iterate through a list of dicts
def fill_name(list_of_dicts, code_name):
    """fill_name takes a list of dicts with keys 'code' and 'name',
    and a dict that maps each code to the correct name as input.  
    This function iterates through the list, and replaces each
    empty name string with the correct name.  
    This function returns a list of dicts."""
    
    # initialize an empty list
    l_out = []
    
    # Iterate over the list of dicts
    for d in list_of_dicts:
        if d['name'] == '':
            d['name'] = code_name[d['code']]
        l_out.append(d)
    
    # return the new list with the correct names
    return l_out           

In [ ]:
# fill in the empty strings in the DataFrame column mjtheme_namecode
df_3['mjtheme_namecode'] = df_3['mjtheme_namecode'].apply(fill_name,args=(dict_mjthemes,))